# Install Package

In [ ]:
# 아래 환경 설치 후 반드시 런타임 재시작 할것
!pip install transformers
!pip install tensorflow_hub

# Import Module

In [2]:
# 기본 패키지
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# 정규화 패키지
import re

# 모델링 패키지
import tensorflow as tf
import transformers
import tensorflow_hub as hub
from transformers import GPT2Tokenizer, TFGPT2Model, BertTokenizer, TFBertModel
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Input, Dropout, Lambda
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# NLP 패키지
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고무시
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [3]:
df = pd.read_csv('Petition_data_english.csv')
df = df.loc[:, ['content_eng', 'label']]
df.head()

,content_eng,label
0,The development of petition AI image generator...,1
1,A petition for severe punishment and disclosur...,1
2,Do you know that the petition government on th...,1
3,KEPCO's petition against privatization of publ...,1
4,I entered OO Elementary School in 2003 and tra...,1


# Data Preprocessing

In [4]:
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    
    # 특수문자 제거
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    # 숫자 제거
    text = re.sub('\w*\d\w*', '', text)
    
    # 불용어 처리
    words = text.split()
    words = [word for word in words if not word in stop_words]
    text = " ".join(words)
    
    return text
  
df['content_eng'] = df['content_eng'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
texts = df["content_eng"].tolist()
labels = df["label"].tolist()

In [6]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
tokenized_texts = [tokenizer.encode(text, add_special_tokens=True) for text in texts]
max_length = max(len(text) for text in tokenized_texts)
padded_texts = tf.keras.preprocessing.sequence.pad_sequences(tokenized_texts, maxlen=max_length, padding='post', truncating='post')

Token indices sequence length is longer than the specified maximum sequence length for this model (603 > 512). Running this sequence through the model will result in indexing errors


#### Train, Test 분리

In [7]:
# 훈련, 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_texts, labels, test_size=0.2, random_state=42)

In [8]:
# 레이블 크기 변경
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

# Modeling

In [10]:
# 모델링
bert_model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)
bert_model.trainable = False # 미세조정하지 않음

input_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)
bert_output = bert_model(input_layer)[0]
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(bert_output)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

earlystopping = EarlyStopping(monitor="val_loss", patience = 5)

checkpoint = ModelCheckpoint('BERT_eng.h5',
                             save_best_only=True,
                             save_weights_only=True,
                             monitor='val_loss',
                             mode='min',
                             verbose=False)

# 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, callbacks=[checkpoint, earlystopping], batch_size=32, validation_data=(X_test, y_test))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

Epoch 1/50
14/14 [==============================] - 102s 6s/step - loss: 0.4927 - accuracy: 0.8272 - val_loss: 0.5646 - val_accuracy: 0.7906
Epoch 2/50
14/14 [==============================] - 93s 7s/step - loss: 0.4847 - accuracy: 0.8268 - val_loss: 0.5358 - val_accuracy: 0.7921
Epoch 3/50
14/14 [==============================] - 95s 7s/step - loss: 0.4698 - accuracy: 0.8277 - val_loss: 0.5133 - val_accuracy: 0.7942
Epoch 4/50
14/14 [==============================] - 94s 7s/step - loss: 0.4701 - accuracy: 0.8292 - val_loss: 0.5213 - val_accuracy: 0.7925
Epoch 5/50
14/14 [==============================] - 94s 7s/step - loss: 0.4715 - accuracy: 0.8295 - val_loss: 0.5177 - val_accuracy: 0.7928
Epoch 6/50
14/14 [==============================] - 95s 7s/step - loss: 0.4696 - accuracy: 0.8295 - val_loss: 0.5131 - val_accuracy: 0.7937
Epoch 7/50
14/14 [==============================] - 94s 7s/step - loss: 0.4652 - accuracy: 0.8296 - val_loss: 0.5202 - val_accuracy: 0.7941
Epoch 8/50
14/14 [=

In [11]:
# 저장된 checkpoint 로드 후 정확도 측정
model.load_weights('BERT_eng.h5')
model.evaluate(X_test, y_test)

4/4 [==============================] - 16s 4s/step - loss: 0.5045 - accuracy: 0.7960


[0.5045055150985718, 0.796010434627533]

In [12]:
# 예측
model.predict(X_test)[0]

4/4 [==============================] - 19s 4s/step


array([[0.16033554],
       [0.13174765],
       [0.14754057],
       ...,
       [0.16004488],
       [0.16004488],
       [0.16004488]], dtype=float32)